In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pip install contractions

In [ ]:
pip install fastext

### Importing Libraries ###

In [131]:
import regex as re
import nltk
import string
import fasttext
import contractions
import nltk
nltk.download('omw-1.4')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
import emoji
import matplotlib as plt
from textblob import TextBlob
from tqdm.notebook import tqdm_notebook
import pandas as pd
import numpy as np
#plt.pyplot.xticks(rotation=70)
pd.options.mode.chained_assignment = None
pd.set_option('display.max_colwidth', 100)
%matplotlib inline

### Reading File ###

In [ ]:
df=pd.read_csv('../input/jigsaw-unintended-bias-in-toxicity-classification/train.csv')

In [ ]:
df.info()

### Removing Duplicates ###

In [ ]:
df=df.drop_duplicates(subset=['comment_text','target'])
df['comment_text']=df['comment_text'].astype('string')

In [ ]:
df.info()

In [ ]:
pretrained_model = "../input/lid-language/lid.176.bin" 
model = fasttext.load_model(pretrained_model)
langs = []
for sent in df['comment_text']:
    lang = model.predict(sent)[0]
    langs.append(str(lang)[11:13])
df['langs'] = langs

In [ ]:
df['langs']=df['langs'].astype('string')
df=df[df['langs']=='en']

In [ ]:
comments=pd.DataFrame(df['comment_text'])
comments.head()

### Dividing dataframe ###

In [ ]:
df_part1=df.iloc[:300000,]
df_part2=df.iloc[300000:600000,]
df_part3=df.iloc[600000:900000,]
df_part4=df.iloc[900000:1200000,]
df_part5=df.iloc[1200000:1500000,]
df_part6=df.iloc[1500000:,]

### Text cleaning function ###

In [126]:
def preprocess_text(text):
    words=[contractions.fix(word) for word in text.split()]
    text=(' ').join(words)
    text=text.lower()
    #print(text)
    text=re.sub(r'https:\S*','',text) #http
    text=re.sub(r'sh\*tty','shitty',text) 
    text=re.sub(r'(ha){2,}','laughing',text) #hahaha
    text=re.sub(r'[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+',"",text) #email
    text=re.sub(r'\S+\.com','',text) #.com
    emoji.demojize(text, delimiters=("", ""))
    text=re.sub(r'\n','',text)
    text=re.sub(r'\-+','',text) #--
    text=re.sub(r'btw','by the way',text)
    text=re.sub(r'\bla\b','',text) #la la la
    text=re.sub(r'<.*?>','',text) #<Fgfdrghd>
    text=re.sub(r'\[','',text) 
    text=re.sub(r'\]','',text)
    text=re.sub(r'hm+','',text) #hmmmm
    text=re.sub(r'mm+','',text) #mmmmm
    text=re.sub(r's\*\*\*\*-storm','shit-storm',text)
    text=re.sub(r'p\*\*\*y','pussy',text)
    text=re.sub(r'f\*cks','fucks',text)
    text=re.sub(r'f\*\*k','fucking',text)
    text=re.sub(r'a\S*\**\S*\*hole','asshole',text)
    text=re.sub(r'a\S*\**\S*\*e','asshole',text)
    text=re.sub(r'c\*\*nt','cunt',text)
    text=re.sub(r'moth\S+\*+s','mother fuckers',text)
    text=re.sub(r'moth\S+\*+r','mother fucker',text)
    text=re.sub(r'b\**\S*\**h','bitch',text)
    text=re.sub(r'n\**\S*\**r','nigger',text)
    text=re.sub(r'b\*+d','bastard',text)
    text=re.sub(r'd\*vilish','devilish',text)
    text=re.sub(r'sh\*t','shit',text)
    text=re.sub(r'f\*+n','fucking',text)
    text=re.sub(r'fuckin','fucking',text)
    text=re.sub(r'dipsh\*t','dipshit',text)
    text=re.sub(r'motha\*+','mother fucking',text)
    text=re.sub(r'stoopiidd','stupid',text)
    text=re.sub(r'p\*+s','pussies',text)
    text=re.sub(r'd\*ck','dick',text)
    text=re.sub(r'f\**\S*\**face','fuck face',text)
    text=re.sub(r'lo+ng','long',text)
    text=re.sub(r'sh\*\*pile','shitpile',text)
    text=re.sub(r'b\*lls','balls',text)
    text=re.sub(r'pr\*ck','prick',text)
    text=re.sub(r'bats\*\*t','batshit',text)
    text=re.sub(r'\'em','asshole',text)
    text=re.sub(r'h\*ll','hell',text)
    text=re.sub(r'gtfo','get the fuck off',text)
    text=re.sub(r'fellas','fellows',text)
    text=re.sub(r'tw\*t','twat',text)
    text=re.sub(r'wh\*re','whore',text)
    text=re.sub(r'imho','in my humble opinion',text)
    text=re.sub(r'f\*\&\^\?','fuck',text)
    text=re.sub(r'(\w)\1{3,}',r'\1',text)   
    text=re.sub(r'[\!\"\#\$\%\&\\\'\(\)\*\+\,\-\.\/\:\;\<\=\>\?\@\[\\\]\^\_\`\{\|\}\~]','',text)
    text=re.sub(r'[0-9]','',text)
    text=re.sub(r'\s\w\s','',text)
    
    text=str(TextBlob(text).correct())
    stop_words = set(stopwords.words('english'))
    words=[word for word in text.split() if word not in stop_words]
    text=(' ').join(words)
    pos=nltk.pos_tag(word_tokenize(text)) 
    def get_wordnet_pos(tag):
        if tag.startswith('J'):
            return wordnet.ADJ
        elif tag.startswith('V'):
            return wordnet.VERB
        elif tag.startswith('N'):
            return wordnet.NOUN
        elif tag.startswith('R'):
            return wordnet.ADV
        else:
            return wordnet.NOUN
    wn_pos=[(word, get_wordnet_pos(pos_tag)) for (word, pos_tag) in pos]
    wnl = WordNetLemmatizer()
    words=[wnl.lemmatize(word, tag) for word, tag in wn_pos]
    text=(' ').join(words)
    return text

In [130]:
tqdm_notebook.pandas()
df_part1['proceesed_comment']=df_part1['comment_text'].apply(prepocess_text)
df_part2['proceesed_comment']=df_part2['comment_text'].apply(prepocess_text)
df_part3['proceesed_comment']=df_part3['comment_text'].apply(prepocess_text)
df_part4['proceesed_comment']=df_part4['comment_text'].apply(prepocess_text)
df_part5['proceesed_comment']=df_part5['comment_text'].apply(prepocess_text)
df_part6['processed_comment']=df_part6['comment_text'].progress_apply(preprocess_text)